In [46]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
data = pd.read_csv("tmdb_5000_movies.csv", low_memory=False)

In [25]:
data.head(10)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-01,890871626,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576
6,260000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-11-24,591794936,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330
7,280000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://marvel.com/movies/movie/193/avenge

In [26]:
data.shape

(4803, 20)

## 데이터 전처리
id, genres,keyword ,voting_average, vote_count, popularity, title, overview

In [27]:
data = data[['id','title','genres','keywords','vote_average','vote_count','popularity','overview']]

In [28]:
data.head(10)

,id,title,genres,keywords,vote_average,vote_count,popularity,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",7.2,11800,150.437577,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",6.9,4500,139.082615,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",6.3,4466,107.376788,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",7.6,9106,112.312950,Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",6.1,2124,43.926995,"John Carter is a war-weary, former military ca..."
5,559,Spider-Man 3,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...","[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",5.9,3576,115.699814,The seemingly invincible Spider-Man goes up ag...
6,38757,Tangled,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",7.4,3330,48.681969,When the kingdom's most wanted-and most charmi...
7,99861,Avengers: Age of Ultron,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 8828, ""name"": ""marvel comic""}, {""id"": ...",7.3,6767,134.279229,When Tony Stark tries to jumpstart a dormant p...
8,767,Harry Potter and the Half-Blood Prince,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 616, ""name"": ""witch""}, {""id"": 2343, ""n...",7.4,5293,98.885637,"As Harry begins his sixth year at Hogwarts, he..."
9,209112,Batman v Superman: Dawn of Justice,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 7002...",5.7,7004,155.790452,Fearing the actions of a god-like Super Hero l...


In [29]:
m = data['vote_count'].quantile(0.9)
data = data.loc[data['vote_count'] >= m]

In [30]:
data.head()

,id,title,genres,keywords,vote_average,vote_count,popularity,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",7.2,11800,150.437577,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",6.9,4500,139.082615,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",6.3,4466,107.376788,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",7.6,9106,112.312950,Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",6.1,2124,43.926995,"John Carter is a war-weary, former military ca..."


In [31]:
data.shape

(481, 8)

In [32]:
C = data['vote_average'].mean()

In [33]:
print(C)
print(m)

6.9629937629937615
1838.4000000000015


In [34]:
def weight_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return (v/ (v+m) * R) + (m/(m+v)*C)

In [35]:
data['score'] = data.apply(weight_rating, axis = 1)

In [36]:
data.head()

,id,title,genres,keywords,vote_average,vote_count,popularity,overview,score
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",7.2,11800,150.437577,"In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",6.9,4500,139.082615,"Captain Barbossa, long believed to be dead, ha...",6.918271
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",6.3,4466,107.376788,A cryptic message from Bond’s past sends him o...,6.493333
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",7.6,9106,112.312950,Following the death of District Attorney Harve...,7.492998
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",6.1,2124,43.926995,"John Carter is a war-weary, former military ca...",6.500396


In [37]:
data.shape

(481, 9)

In [38]:
data['genres'] = data['genres'].apply(ast.literal_eval)
data['keywords'] = data['keywords'].apply(ast.literal_eval)

In [39]:
data.head()

,id,title,genres,keywords,vote_average,vote_count,popularity,overview,score
0,19995,Avatar,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':...",7.2,11800,150.437577,"In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,Pirates of the Caribbean: At World's End,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",6.9,4500,139.082615,"Captain Barbossa, long believed to be dead, ha...",6.918271
2,206647,Spectre,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",6.3,4466,107.376788,A cryptic message from Bond’s past sends him o...,6.493333
3,49026,The Dark Knight Rises,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",7.6,9106,112.312950,Following the death of District Attorney Harve...,7.492998
4,49529,John Carter,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':...",6.1,2124,43.926995,"John Carter is a war-weary, former military ca...",6.500396


In [40]:
data['genres'] = data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x: " ".join(x))
data['keywords'] = data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x: " ".join(x))

In [41]:
data.head()

,id,title,genres,keywords,vote_average,vote_count,popularity,overview,score
0,19995,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,7.2,11800,150.437577,"In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,6.9,4500,139.082615,"Captain Barbossa, long believed to be dead, ha...",6.918271
2,206647,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6 bri...,6.3,4466,107.376788,A cryptic message from Bond’s past sends him o...,6.493333
3,49026,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,7.6,9106,112.312950,Following the death of District Attorney Harve...,7.492998
4,49529,John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,6.1,2124,43.926995,"John Carter is a war-weary, former military ca...",6.500396


In [42]:
count_vector = CountVectorizer(ngram_range=(1,3))

In [43]:
C_vector_genres = count_vector.fit_transform(data['genres'])

In [45]:
C_vector_genres.shape

(481, 364)

In [47]:
gerne_c_sim = cosine_similarity(C_vector_genres, C_vector_genres).argsort()[:, ::-1]

In [48]:
gerne_c_sim.shape

(481, 481)

In [49]:
def get_recommend_movie_list(df, movie_title, top=32):
    target_movie_index = df[df['title'] == movie_title].index.values
    
    sim_index = gerne_c_sim[target_movie_index, :top].reshape(-1)
    
    sim_index = sim_index[sim_index != target_movie_index]
    
    result = df.iloc[sim_index].sort_values('score', ascending=False)[:10]
    return result

In [50]:
get_recommend_movie_list(data, movie_title='The Dark Knight Rises')

,id,title,genres,keywords,vote_average,vote_count,popularity,overview,score
65,155,The Dark Knight,Drama Action Crime Thriller,dc comics crime fighter secret identity scarec...,8.2,12002,187.322927,Batman raises the stakes in his war on crime. ...,8.035690
2091,274,The Silence of the Lambs,Crime Drama Thriller,based on novel psychopath horror suspense seri...,8.1,4443,18.174804,"FBI trainee, Clarice Starling ventures into a ...",7.767228
2760,264644,Room,Drama Thriller,based on novel carpet isolation kidnapping imp...,8.1,2757,66.113340,Jack is a young boy of 5 years old who has liv...,7.645138
351,1422,The Departed,Drama Thriller Crime,undercover boston police friends mafia underco...,7.9,4339,63.429157,"To take down South Boston's Irish Mafia, the p...",7.621146
1850,111,Scarface,Action Crime Drama Thriller,miami corruption capitalism cuba prohibition b...,8.0,2948,70.105981,After getting a green card in exchange for ass...,7.601698
4337,103,Taxi Driver,Crime Drama,vietnam veteran taxi obsession drug dealer nig...,8.0,2535,58.845025,A mentally unstable Vietnam War veteran works ...,7.564085
1051,146233,Prisoners,Drama Thriller Crime,pennsylvania kidnapping maze vigilante rural s...,7.9,3085,88.496873,When Keller Dover's daughter and her friend go...,7.550121
828,24,Kill Bill: Vol. 1,Action Crime,japan coma martial arts kung fu underworld yak...,7.7,4949,79.754966,An assassin is shot at the altar by her ruthle...,7.500378
3701,641,Requiem for a Dream,Crime Drama,drug addiction junkie heroin speed diet unsoci...,7.9,2443,11.573034,The hopes and dreams of four ambitious people ...,7.497657
3940,670,Oldboy,Drama Thriller Mystery Action,sushi restaurant rage and hate notebook daught...,8.0,1945,56.763490,"With no clue how he came to be imprisoned, dru...",7.496106
